<a href="https://colab.research.google.com/github/ikazreal21/machinelearning_sklearn_pytorch/blob/master/pytorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
!unzip kagglecatsanddogs_3367a.zip
!pip install numpy opencv-python tqdm
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

###CODE


In [ ]:
import os 
import numpy as np
import cv2 
from tqdm import tqdm 
# from google.colab.patches import cv2_imshow 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



REBUILD_DATA = False

class DogsvCats():
    IMG_SIZE = 50
    CATS = "/content/PetImages/Cat"
    DOGS = "/content/PetImages/Dog"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []
    catcnt = 0
    dogcnt = 0


    def making_training_data(self):
        for label in self.LABELS:
            print(label)
            for d in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, d)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])   

                    if label == self.CATS:
                        self.catcnt += 1
                    elif label == self.DOGS:
                        self.dogcnt += 1
                except  Exception as e:
                    pass    

        np.random.shuffle(self.training_data)
        np.save("training_dats.npy", self.training_data)

        # print("Cats: ", self.catcnt)
        # print("Dogs: ", self.dogcnt)

# plt.imshow(training_data[4][0], cmap="gray")
# plt.show()
# training_data[4][1]


class Cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.conn1 = nn.Conv2d(1, 32, 5)
        self.conn2 = nn.Conv2d(32, 64, 5)
        self.conn3 = nn.Conv2d(64, 128, 5)
        self.conn4 = nn.Conv2d(128, 256, 5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.conv(x)

        self.fconn1 = nn.Linear(self._to_linear, 512)
        self.fconn2 = nn.Linear(512, 2)

    def conv(self, x):
        x = F.max_pool2d(F.relu(self.conn1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conn2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conn3(x)), (2,2))
        # x = F.max_pool2d(F.relu(self.conn4(x)), (2,2))


        # print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fconn1(x))
        x = self.fconn2(x)
        return F.softmax(x, dim=1)

net = Cnn()


if REBUILD_DATA:
    dogsvcats = DogsvCats()
    dogsvcats.making_training_data()
    
training_data = np.load("training_dats.npy", allow_pickle=True)
print(len(training_data))

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_pct = 0.1

valsize = int(len(X)*VAL_pct)
print(valsize)

train_X = X[:-valsize]
train_y = y[:-valsize]

test_X = X[-valsize:]
test_y = y[-valsize:]

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("On GPU")
else: 
    device = torch.device("cpu")
    print("On CPU")

net = Cnn().to(device)

In [ ]:
def forward_pass(X, y, train=False):
    if train:
        net.zero_grad()
    outputs = net(X.view(-1,1,50,50).to(device))
    matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_func(outputs, y.to(device))

    if train:
        loss.backward()
        optimizer.step()
    return acc, loss

In [ ]:
def test(size=32):
    random_s = np.random.randint(len(test_X)-size)
    X, y = test_X[random_s:random_s+size], test_y[random_s:random_s+size]
    with torch.no_grad():
        val_acc, val_loss = forward_pass(X, y)
    return val_acc, val_loss

val_acc, val_loss = test(size=1000)
print(val_acc, val_loss)

0.5 tensor(0.2501, device='cuda:0')


In [ ]:
import time 

Model_name = f"model-{int(time.time())}"

net = Cnn().to(device)

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.MSELoss()

print(f"\n{Model_name}")

def train():
    BATCH_SIZE = 100
    epochs = 30

    with open("model.log", "a") as f:
        for epchs in range(epochs):
            for d in tqdm(range(0, len(train_X), BATCH_SIZE)):
                # print(d, d+BATCH_SIZE)
                batch_X = train_X[d:d+BATCH_SIZE].view(-1, 1, 50, 50).to(device) 
                batch_y = train_y[d:d+BATCH_SIZE].to(device)

                acc, loss = forward_pass(batch_X, batch_y, train=True)
                if d % 30 == 0:
                    val_acc, val_loss = test(size=100)
                    f.write(f"{Model_name}, {round(time.time(),3)}, {round(float(acc),2)}, {round(float(loss),4)}, {round(float(val_acc),2)},{round(float(val_loss),4)}\n")
train()